In [0]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Usar Spark para cargar la tabla en un DataFrame de Spark
df_spark = spark.table("default.saber_limpio_csv")

# Convertir el DataFrame de Spark a un DataFrame de Pandas
df_SaberPro = df_spark.toPandas()

# Mostrar los primeros registros del DataFrame resultante
print(df_SaberPro.head())

columnas_interes = ['FAMI_ESTRATOVIVIENDA','FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE','FAMI_TIENEAUTOMOVIL','FAMI_TIENECOMPUTADOR','FAMI_TIENEINTERNET','FAMI_TIENELAVADORA']

#label_encoder = LabelEncoder()
#for columna in columnas_interes:
#    df_Saber11[columna] = label_encoder.fit_transform(df_Saber11[columna])

label_encoder = LabelEncoder()
for columna in columnas_interes:
    df_SaberPro[columna] = label_encoder.fit_transform(df_SaberPro[columna])

# Preparar los datos
y = df_SaberPro['PUNT_GLOBAL']  # Variable objetivo
X = df_SaberPro.drop(columns=['PUNT_GLOBAL'])  # Resto de las columnas

# Seleccionar solo las columnas numéricas
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
X_numerical = X[numerical_columns]

# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numerical)

# Reducir las dimensiones con PCA
pca = PCA(n_components=6)  
X_pca = pca.fit_transform(X_scaled)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# registre el experimento
experiment = mlflow.set_experiment("/Users/s.barrera@uniandes.edu.co/Saber_11_MFLOW")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # defina los parámetros del modelo
    n_estimators = 50 
    max_depth = 2
    max_features = 2
    # Cree el modelo con los parámetros definidos y entrénelo
    rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
    rf.fit(X_train, y_train)
    # Realice predicciones de prueba
    predictions = rf.predict(X_test)
  
    # Registre los parámetros
    mlflow.log_param("num_trees", n_estimators)
    mlflow.log_param("maxdepth", max_depth)
    mlflow.log_param("max_feat", max_features)
  
    
    # Registrar el modelo con ejemplo de entrada
    mlflow.sklearn.log_model(rf, "random-forest-model")
    # Cree y registre la métrica de interés
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)
    print(mse)


com.databricks.backend.daemon.driver.DriverClientDestroyedException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$4(DriverClient.scala:754)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$2(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:271)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:267)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.threading.ContextBoundRunnable.withAttributionContext(ContextBoundRunnable.scala:6)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$1(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:73)
	at com.databricks.threading.ContextBoundRunnable.run(ContextBoundRunnable.scala:14)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:477)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)